[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/psychoanalyze/psychoanalyze/main?labpath=docs%2Fnotebooks%2Ftutorial.ipynb)
# PsychoAnalyze Python Tutorial / Demo

## Ordinary Least Squares Logistic Regression on Trial Data

### Data Schema

The simplest psychophysical experiments consist of a series of trials measuring a binary-encoded response to an intensity-graded stimulus.

Use the `trials` module to load data from a CSV or Parquet file into a Pandas DataFrame.

In [3]:
from psychoanalyze.data import trials

example_trials = trials.load("tutorial_trials.csv")
example_trials


,Block,Intensity,Result
0,0,0.0,0
1,0,1.0,0
2,0,2.0,1
3,0,3.0,1
4,1,0.0,0
5,1,1.0,0
6,1,2.0,1
7,1,3.0,1


In [4]:
example_trials.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8 entries, 0 to 7
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Block      8 non-null      int64  
 1   Intensity  8 non-null      float64
 2   Result     8 non-null      int64  
dtypes: float64(1), int64(2)
memory usage: 324.0 bytes


### Logistic Regression in a Yes/No Experiment

By designing the experiment such that the recorded response is a binary outcome, researchers may use logistic regression to model the relationship between the stimulus and the response; logistic regression is a generalized linear model that is appropriate for a series of binary (Bernoulli) trials.

Let's examine the 1-dimensional case of the logistic function, which may represent a basic psychometric function $\psi(x)$, using some tools from `psychoanalyze`.  $\psi(x)$ models the probability of a "yes" response to a stimulus of intensity *x*:

In [6]:
from psychoanalyze.data.blocks import standard_logistic
from psychoanalyze import plot
import pandas as pd
import plotly.express as px
import plotly.io as pio
from dash import Dash, dcc, html, callback, Input, Output
import dash_bootstrap_components as dbc

pio.renderers.default = "plotly_mimetype+notebook_connected"

app = Dash(__name__, external_stylesheets=[dbc.themes.SPACELAB])

app.layout = dbc.Container(
    dbc.Row(
        [
            dbc.Col(
                html.Button(
                    "Standard Logistic",
                    id="standard-logistic-button",
                    className="mb-3 btn btn-primary",
                ),
            ),
            dbc.Col(
                [
                    html.H5("Location"),
                    dcc.Slider(
                        min=-4.0,
                        max=4.0,
                        step=0.01,
                        value=0.0,
                        included=False,
                        tooltip={"always_visible": True, "placement": "bottom"},
                        updatemode="drag",
                        marks={
                            -4.0: "-4.0",
                            -2.0: "-2.0",
                            0.0: "0.0",
                            2.0: "2.0",
                            4.0: "4.0",
                        },
                        id="location-slider",
                    ),
                ]
            ),
            dbc.Col(
                [
                    html.H5("Scale"),
                    dcc.Slider(
                        min=-4.0,
                        max=4.0,
                        value=1.0,
                        step=0.01,
                        included=False,
                        marks={
                            -4.0: "-4.0",
                            -2.0: "-2.0",
                            0.0: "0.0",
                            2.0: "2.0",
                            4.0: "4.0",
                        },
                        tooltip={"always_visible": True, "placement": "bottom"},
                        updatemode="drag",
                        id="scale-slider",
                    ),
                ]
            ),
            dcc.Graph(
                figure=plot.logistic(location=0.0, scale=1.0),
                id="psi-plot",
            ),
        ]
    )
)


@callback(
    Output("psi-plot", "figure"),
    Input("location-slider", "value"),
    Input("scale-slider", "value"),
)
def update_psi_plot(location, scale):
    return plot.logistic(location=location, scale=scale)


if __name__ == "__main__":
    app.run(debug=True)
